<a href="https://colab.research.google.com/github/FATE4869/freeMAML/blob/master/caleb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import click
import os, sys
import numpy as np
import random
# from setproctitle import setproctitle
import inspect
import pdb
from collections import OrderedDict
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Parameter
from torch.optim import SGD, Adam
from torch.nn.modules.loss import CrossEntropyLoss

from task import OmniglotTask, MNISTTask
from dataset import Omniglot, MNIST
from inner_loop import InnerLoop
# from omniglot_net import OmniglotNet
from score import *
from data_loading import *
import torchvision
import numpy as np
import random
import math
from collections import OrderedDict

import torch
import torch.nn as nn
import copy
from layers import *


In [0]:
class OmniglotNet(nn.Module):
    '''
    The base model for few-shot learning on Omniglot
    '''

    def __init__(self, num_classes, loss_fn, num_in_channels=3):
        super(OmniglotNet, self).__init__()
        # Define the network
        self.features = nn.Sequential(OrderedDict([
                ('conv1', nn.Conv2d(num_in_channels, 64, 3)),
                ('bn1', nn.BatchNorm2d(64, momentum=1, affine=True)),
                ('relu1', nn.ReLU(inplace=True)),
                ('pool1', nn.MaxPool2d(2,2)),
                ('conv2', nn.Conv2d(64,64,3)),
                ('bn2', nn.BatchNorm2d(64, momentum=1, affine=True)),
                ('relu2', nn.ReLU(inplace=True)),
                ('pool2', nn.MaxPool2d(2,2)),
                ('conv3', nn.Conv2d(64,64,3)),
                ('bn3', nn.BatchNorm2d(64, momentum=1, affine=True)),
                ('relu3', nn.ReLU(inplace=True)),
                ('pool3', nn.MaxPool2d(2,2))
                # ('conv4', nn.Conv2d(64,64,3)),
                # ('bn4', nn.BatchNorm2d(64, momentum=1, affine=True)),
                # ('relu4', nn.ReLU(inplace=True)),
                # ('pool4', nn.MaxPool2d(2,2))
        ]))
        self.add_module('fc', nn.Linear(64, num_classes))
        
        # Define loss function
        self.loss_fn = loss_fn

        # Initialize weights
        self._init_weights()

    def forward(self, x, weights=None):
        ''' Define what happens to data in the net '''
        if weights == None:
            
            x = self.features(x)
            x = x.view(x.size(0), 64)
            x = self.fc(x)
        else:
            x = conv2d(x, weights['features.conv1.weight'], weights['features.conv1.bias'])
            x = batchnorm(x, weight = weights['features.bn1.weight'], bias = weights['features.bn1.bias'], momentum=1)
            x = relu(x)
            x = maxpool(x, kernel_size=2, stride=2) 
            x = conv2d(x, weights['features.conv2.weight'], weights['features.conv2.bias'])
            x = batchnorm(x, weight = weights['features.bn2.weight'], bias = weights['features.bn2.bias'], momentum=1)
            x = relu(x)
            x = maxpool(x, kernel_size=2, stride=2) 
            x = conv2d(x, weights['features.conv3.weight'], weights['features.conv3.bias'])
            x = batchnorm(x, weight = weights['features.bn3.weight'], bias = weights['features.bn3.bias'], momentum=1)
            x = relu(x)
            x = maxpool(x, kernel_size=2, stride=2) 
            # x = conv2d(x, weights['features.conv4.weight'], weights['features.conv4.bias'])
            # x = batchnorm(x, weight = weights['features.bn4.weight'], bias = weights['features.bn4.bias'], momentum=1)
            # x = relu(x)
            # x = maxpool(x, kernel_size=2, stride=2) 
            x = x.view(x.size(0), 64)
            x = linear(x, weights['fc.weight'], weights['fc.bias'])

        return x

    def net_forward(self, x, weights=None):
        return self.forward(x, weights)
    
    def _init_weights(self):
        ''' Set weights to Gaussian, biases to zero '''
        torch.manual_seed(1337)
        torch.cuda.manual_seed(1337)
        torch.cuda.manual_seed_all(1337)
        # print ('init weights')
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                n = m.weight.size(1)
                m.weight.data.normal_(0, 0.01)
                #m.bias.data.zero_() + 1
                m.bias.data = torch.ones(m.bias.data.size())
    
    def copy_weights(self, net):
        ''' Set this module's weights to be the same as those of 'net' '''
        # TODO: breaks if nets are not identical
        # TODO: won't copy buffers, e.g. for batch norm
        for m_from, m_to in zip(net.modules(), self.modules()):
            if isinstance(m_to, nn.Linear) or isinstance(m_to, nn.Conv2d) or isinstance(m_to, nn.BatchNorm2d):
                m_to.weight.data = m_from.weight.data.clone()
                if m_to.bias is not None:
                    m_to.bias.data = m_from.bias.data.clone()


In [135]:
import torch
import torchvision
from torch.utils.data import Dataset,DataLoader,sampler
# from torch.utils.data.sampler import SubsetRandomSampler
dataset = torchvision.datasets.Omniglot(
    root="./data", download=True, transform=torchvision.transforms.ToTensor()
)
image, label = dataset[0]
# print(type(image))  # torch.Tensor
# print(type(label))  # int
for i in range(10):
    image, label = dataset.__getitem__(i)
    # print(image.shape)
    # print(label)
data_loader = DataLoader(dataset, batch_size=64, shuffle=True)
data_iter = iter(data_loader)
for i in range(10):
    # print(i)
    image,label = next(data_iter)
    # print(image.shape)
    # print(label)

Files already downloaded and verified


In [0]:
def get_task():
  root = './data/omniglot-py'
  split='train'
  n_cl = 20
  n_inst = 1
  return OmniglotTask(root, n_cl, n_inst, split)

def Hvp(net, batch_X, batch_y, v):
    """
    Computes an Hessian-vector product using the Pearlmutter method.
    The Hessian used is the loss function of tf.keras.Model 'model' with respect to its trainable_variables.
    The loss is evaluated on the given batch.
    The vector v is decomposed into a list of Tensors, to match the number of weights of 'model' and their shape.
    """
    loss, _ = net.forward_pass(batch_X, batch_y)
    # for name, param in net.parameters():
    for name, param  in net.parameters():
      print(name, param.shape)
    grad = torch.autograd.grad(loss, net.parameters())
    
    for name, param  in net.parameters():
      print(name, param.shape)
    Hv = torch.autograd.grad(grad, net.parameters(), v)

    return Hv

def grads_on_batch(net, batch_X, batch_y):
    loss, _ = net.forward_pass(batch_X, batch_y)

    grads = torch.autograd.grad(loss, net.parameters(), create_graph=True)
    return grads

# Optimizers that can be used to approximate the meta-gradient (I+1/lambda*H)*g = b -> Ag = b
# A is supplied as a linear operator function, taking a vector as argument and returning the matrix-vector product.
def plain_gradient_descent(Av, b, x0, num_iterations, learning_rate=0.01, debug=False):
    ## Gradient descent
    for i in range(num_iterations):
        Ax = Av(x0)
        for k in range(len(x0)):
            x0[k] = x0[k] - 0.01 * (Ax[k]-b[k])

        if debug:
            print(f1(x0, Av, b), f2(x0, Av, b) , '\n\n')

    return x0


class InnerLoop(OmniglotNet):
    
    def __init__(self, num_classes, loss_fn, num_updates, step_size, batch_size, meta_batch_size, num_in_channels = 3):
        super(InnerLoop, self).__init__(num_classes, loss_fn, num_in_channels)

        # Number of updates to be taken
        self.num_updates = num_updates

        # Step size for the update
        self.batch_size = meta_batch_size

        # Per class batch size for the updates
        self.batch_size = batch_size

        # for loss normalization
        self.meta_batch_size = meta_batch_size

        # sth
        self.n_iters_optimizer = 1
        # print(self.parameters())
    def initialize(self):
        """
        This method should be called after the wrapped model is compiled.
        """
        self.current_initial_parameters = [v for (k,v) in net.named_parameters() ]
        return self.current_initial_parameters

    def task_begin(self, current_initial_parameters, task = None, **kwargs):
        with torch.no_grad():

          count = 0
          for name, param in net.named_parameters():
            param.copy_(current_initial_parameters[count])
            count += 1
         
    def task_end(task=None, **kwargs):
        """
        Method to call after training on each meta-batch task; possibly return relevant information for the
        meta-learner to use for the meta-updates.
        """
        assert task is not None, "FOMAML needs a `task' argument on .task_end to compute the data it needs."

        # Compute the gradient of the TEST LOSS evaluated at the final parameters. E.g., mean gradients
        # over a batch of test data

        ret_grads = gradients_for_task_CG(task)

        return ret_grads
    def forward_pass(self, in_, target, weights=None):
        ''' Run data through net, return loss and output '''
        input_var = torch.autograd.Variable(in_)
        target_var = torch.autograd.Variable(target)
        # Run the batch through the net, compute loss
        out = self.net_forward(input_var, weights)
        loss = self.loss_fn(out, target_var)
        return loss, out

    def gradients_for_task_CG(self, task = 'omniglot'):
        batch = 1
        train_loader = get_data_loader(task, batch, 'train')
        test_x, test_y = train_loader.__iter__().next()
        train_x, train_y = train_loader.__iter__().next()

        def Av(v):
            Hv = Hvp(net, train_x, train_y, v)
            return [v[i] + 1.0/1.0 * Hv[i].numpy() for i in range(len(v))]

        test_gradients = [g.detach().numpy() for g in grads_on_batch(self, test_x, test_y)]

        x0 = [np.zeros(var.shape, dtype = np.float32) for var in self.current_initial_parameters]

        b = copy.deepcopy(test_gradients)

        x = plain_gradient_descent(Av, b, x0, self.n_iters_optimizer, learning_rate=0.01)

        return x
    def update(self, list_of_final_gradients, **kwargs):
        """
        Main (outer-loop) training operation. Call after training on a whole meta-batch, after each meta-iteration
        has finished.
        """
        # Perform a FOMAML update for the outer meta-training iteration. Take the expected test gradient
        # L'(\tilde{phi}) over tasks in the meta-batch and perform a single step of gradient descent.
        avg_final_grads = []
        for grads in zip(*list_of_final_gradients):
            avg_final_grads.append(np.mean(grads, axis=0))

        optimizer = torch.optim.SGD(self.named_parameters(), lr=1e-4)

        # Apply gradients to the *initial parameters*
        for i in range(len(self.current_initial_parameters)):
            self.named_parameters[i].assign( self.current_initial_parameters[i] )
            self.named_parameters[i].grad = avg_final_grads[i]

        
        optimizer.step()

        # Set the new initial parameters
        self.current_initial_parameters = [v.numpy() for v in self.named_parameters] 


In [151]:
dataset='omniglot'
num_classes= 20
num_inst=1
batch_size = 1
meta_batch_size = 16
num_updates=1
num_inner_updates=1
num_in_channels = 3
step_size='1e-1'
inner_lr = 0.1
meta_lr='1e-3'
gpu= 0
input_size = 28
loss_fn = CrossEntropyLoss()

net = InnerLoop(num_classes, loss_fn, num_updates, step_size, batch_size, meta_batch_size, num_in_channels)
current_initial_parameters = net.initialize()
task = get_task()

net.task_begin(current_initial_parameters)

inps = torch.rand(batch_size, num_in_channels, input_size, input_size, requires_grad=False)
targets = torch.randint(0, num_classes, (batch_size,))
# net.train()


# loss, out = net.forward_pass(inps, targets, weights=None)
x = net.gradients_for_task_CG(task)

init weights


ValueError: ignored

In [0]:
x